In [40]:
import re
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/kim-yongjun/Documents/부스트캠프 AI Tech 7기/5.5. NLP 기초 프로젝트/nlp_project/train.csv')


### 1. Label의 균형을 맞춰야 함. 
(Augumentation을 통해 해결)
### 2. 자주 등장하는 'PERSON' 단어를 Tokenizer마다 다르게 해석할 수 있으므로 하나의 Special Token으로 추가해야 함.
(Modeling 단계에서 추가)
### 3. 같은 단어들이 반복되는 문장들을 교정해야 함.
(Preprocessing 단계에서 해결)
### 4. 대문자들을 전부 소문자로 바꿔 혼란을 줄여야 함.
(Preprocessing 단계에서 해결), 추가적으로 '!', '?', '.'을 제외한 특수문자는 삭제하도록 하였음.
### 5. 띄어쓰기 교정을 통해 올바른 문장으로 바꿔야 함.
(Preprocessing 단계에서 해결)
### 6. 오타로 인한 [UNK] Token 생성을 줄여야 함.
(Preprocessing 단계에서 해결)


In [41]:
def reduce_repeated_chr(sentence):
    # 정규식 패턴: 같은 문자가 3번 이상 반복되는 경우 찾기
    pattern = r'(.)\1{2,}'
    return re.sub(pattern, r'\1\1', sentence)

def reduce_repeated_words(sentence):
    # 정규식 패턴: 같은 단어가 3번 이상 반복되는 경우 찾기 (띄워쓰기 포함)
    pattern = r'\b(\w+)\b(?:\s+\1){2,}'
    return re.sub(pattern, r'\1 \1', sentence)
    
def reduce_repeated_phrases(sentence):
    # 정규식 패턴: 같은 단어나 구절이 3번 이상 반복되는 경우 찾기
    pattern = r'(\w+)\1{2,}'
    return re.sub(pattern, r'\1\1', sentence)

sentence = '야호야호야호 냥 냥 냥 냥 냥 ㅎㅎㅎㅎ !!!!!!! ㅋㅋㅋㅋㅋ'
print(sentence)
sentence = reduce_repeated_chr(sentence)
print(sentence)
sentence = reduce_repeated_words(sentence)
print(sentence)
sentence = reduce_repeated_phrases(sentence)
print(sentence)

야호야호야호 냥 냥 냥 냥 냥 ㅎㅎㅎㅎ !!!!!!! ㅋㅋㅋㅋㅋ
야호야호야호 냥 냥 냥 냥 냥 ㅎㅎ !! ㅋㅋ
야호야호야호 냥 냥 ㅎㅎ !! ㅋㅋ
야호야호 냥 냥 ㅎㅎ !! ㅋㅋ


## !!!!!!! 같이 반복되는 단어들을 2자로 줄입니다.

In [42]:
def erase_special_chr(sentence):
    sentence = sentence.lower()
    pattern = r'[^ㄱ-ㅎ가-힣a-zA-Z0-9\s?.!]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

## 문장의 영문들을 소문자로 바꾸고 '!', '.', '?'를 제외한 특수문자를 전부 제거합니다.

In [54]:
import requests
import json

class SpellChecker:
    def __init__(self):
        # 직접 찾아서 넣은 passportKey와 base_url
        self.base_url = 'https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy'
        self.passport_key = '062220e9b6aee9cf862e0ca85fb511078b8f04d0'  # 직접 찾은 passportKey 입력

    def spell_check(self, text):
        # base_url이나 passport_key가 없는 경우 에러 반환
        if self.passport_key is None or self.base_url is None:
            return "Error: Missing base_url or passport_key"
        
        payload = {
            'passportKey': self.passport_key,
            'where': 'nexearch',
            'color_blindness': 0,
            'q': text
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/129.0.0.0 Safari/537.36',
            'Referer': 'https://search.naver.com/',
        }
        result_response = requests.get(self.base_url, headers=headers, params=payload)
        return json.loads(result_response.text)['message']['result']['notag_html']

# Usage
checker = SpellChecker()
sentence = '책을많이읽어봣는데도맞춘법에약해요 ㅋㅋㅋㅋㅋ.'
print(sentence)
result = checker.spell_check(sentence)
print(result)


책을많이읽어봣는데도맞춘법에약해요 ㅋㅋㅋㅋㅋ.
책을 많이 읽어봤는데도 맞춤법에 약해요 ㅋㅋㅋㅋㅋ.


## 맞춤법 교정 라이브러리를 통해 오타와 띄워쓰기를 교정합니다.
## 네이버 맞춤법 검사기를 통해 교정된 문장을 가져옵니다.
## 네이버에 맞춤법 검사기를 검색 후 소스보기에서 ctrl+f , passport를 검색하여 키를 붙여넣습니다.

In [44]:
sentence = 'SENTENCE : ~우물안개구리바다의넓음은 몰라도 하늘에 푸르름은 안다 !!!!!!~ 몬가 멋진 말이죠????? ㅋㅋㅋㅋㅋ'
print(sentence)
sentence = reduce_repeated_chr(sentence)
sentence = reduce_repeated_words(sentence)
sentence = reduce_repeated_phrases(sentence)
print(sentence)
sentence = erase_special_chr(sentence)
sentence = checker.spell_check(sentence)
print(sentence)

SENTENCE : ~우물안개구리바다의넓음은 몰라도 하늘에 푸르름은 안다 !!!!!!~ 몬가 멋진 말이죠????? ㅋㅋㅋㅋㅋ
SENTENCE : ~우물안개구리바다의넓음은 몰라도 하늘에 푸르름은 안다 !!~ 몬가 멋진 말이죠?? ㅋㅋ
sentence  우물 안 개구리 바다의 넓음은 몰라도 하늘에 푸르름은 안다!! 뭔가 멋진 말이죠?? ㅋㅋ


In [45]:
def preprocessing(sentence):
    sentence = reduce_repeated_chr(sentence)
    sentence = reduce_repeated_words(sentence)
    sentence = reduce_repeated_phrases(sentence)
    sentence = erase_special_chr(sentence)
    sentence = checker.spell_check(sentence)
    return sentence

In [47]:
from tqdm import tqdm
tqdm.pandas()
df['sentence_1'] = df['sentence_1'].progress_map(preprocessing)
df['sentence_2'] = df['sentence_2'].progress_map(preprocessing)

100%|███████████████████████████████████████| 9324/9324 [10:43<00:00, 14.48it/s]


## 선언한 함수들을 통합한 preprocessing을 통해 데이터들을 전처리합니다.

In [49]:
df['sentence_1'].sample(20)

3113                 삶에 대해 다시 한번 생각해 볼 수 있는 좋은 시간이었습니다
2093                         별 내용은 없고 여주 몸매는 좋으나 그게 다임
722                                     반려동물 처벌 높여주세요.
5538                                  person 님 정말 팬이에요
4352                            사건이 해결되지 않은 몰카 피해자입니다.
2226                              남은 시간 행복하게 보내고 오세요!!
5780                                       우리도 곧 업 업!!
6275    이재명 전시장 지지자의 악의적 sns 활동의 당과 청와대의 진상 규명을 요구합니다.
6109                                    천안함 재조사 요구합니다!
735                                        사법고시를 폐지하세요
603       작년 여름 에어컨을 6월에 주문했는데 8월 말에 설치했어요 얼른 설치하세요 ㅋㅋ
1827                       국회의원 임금을 최저 임금으로 지급하는 청원입니다
2298                                       박철수 감독의 유작.
872                                     조여옥 대위 처벌해 주세요
5071                                  엘처럼 멋지게 나도 살 겁니다
559                         다음에 태도가 작품이 될 때 리뷰도 올려주세요!
435                                      오오 이쁘네요 여기 ㅎㅎ
4649                                       계속 볼 겁니다 흐흐
4186      

In [50]:
df['sentence_2'].sample(20)

661                  멋진 기세.. 그리고 산드라 블록.. 술과 사랑..
8334                  사진같이 크게 도약하는 한 해가 되시길 바랍니다.
7769                     장난 청원하는 사람들 엄격히 처벌해 주세요.
6129                          남자 강제 군대 징병은 위헌입니다.
2415                                   와 너무 귀여워요 
4172                             청소년 보호법 폐지해 주세요!
4423             차량 구입 시 실사 용주가 차량을 구매하도록 조치 바랍니다
6755       남북 과제의 최우선 항목 중 역사공동조사 부분을 꼭 포함하여 주세요.
1983                      중국발 미세먼지 중국에 꼭 항의해야 합니다
6545                            누구를 위한 대체공휴일인가요? 
84                                   천천히 회복하면서 휴식
8721                                  자연이 역시 최고네요
5708                             과연 믿고 보는 기타노 다케시
6601                                  애들이 많이 불쌍..
2660           sns를 많이 사용하시는 분들에게 좋은 콘텐츠인 것 같습니다.
6676    아파트에서 자동차가 자유롭게 다닐 수 있는 일반 도로를 건설하고 있습니다.
2551                                근데 연재는 완료되었는데
3539                          그래도 이루어졌으면 더 좋았을걸..
7020                     일을 하지 않으려면 세금을 적당히 걷습니다.
7170                              야근 방지 실현화 시켜주세요


## 전처리 후의 데이터는 띄워쓰기나 중복, 오타가 많이 사라진 모습을 보인다.
## 하지만 고유 명사는 맞춤법 교정을 통한 교정이 어려워 보인다. ex) mb > 명박 , 이제명 > 이재명 등
## 모델의 성능을 위해서는 몇몇의 label 이상이 있거나 교정이 어려운 행을 삭제하는 것이 도움이 됨을 알 수 있다.

In [53]:
df.to_csv('preprocessed_data.csv')